In [1]:
import pandas as pd #csv파일 읽어오는 모듈
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.sparse.linalg import svds
from ast import literal_eval


import warnings; warnings.simplefilter('ignore')

In [2]:
def return_recipe_id(recipe_title):
    return md[md['recipe_title']==recipe_title]['recipe_id']

def recommend_recipe(df_svd_preds, user_id, ori_recipe_df, ori_ratings_df, num_recommendations=5):
    
    #현재는 index로 적용이 되어있으므로 user_id - 1을 해야함.
   # user_row_number = user_id - 1 
    target_user = df_user.index(user_id)

    # 최종적으로 만든 pred_df에서 사용자 index에 따라 데이터 정렬 -> 레시피 평점(나중엔 스코어) 높은 순으로 정렬 됌
    sorted_user_predictions = df_svd_preds.iloc[target_user].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_ratings_df[ori_ratings_df.user_token_id == user_id]
    #user_data.head(10)
    
    # 위에서 뽑은 user_data와 원본 레시피 데이터를 합친다. 
    user_history = user_data.merge(ori_recipe_df, on = 'recipe_id')
    user_history.sort_values("values",axis=0, ascending=False)
    
    # 원본 레시피들에서 사용자가 본 메뉴를 제외한 데이터를 추출
    recommendations = ori_recipe_df[~ori_recipe_df['recipe_id'].isin(user_history['recipe_id'])]
    
    alpha = pd.DataFrame(sorted_user_predictions).reset_index().rename(columns={'index':'recipe_id',target_user:'Predictions'})
    alpha=alpha.astype({'recipe_id':'int'})
    # 사용자의 평점이 높은 순으로 정렬된 데이터와 합친다. 
    recommendations = recommendations.merge(alpha, on = 'recipe_id',how='left')
                                            
    # 컬럼 이름 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {user_id: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
                      
    
    return user_history, recommendations#alpha

In [37]:
if __name__ =="__main__":

    recommend_userid =   'test-d80-w20-k10'
    md = pd.read_csv('./yorizori/recipe_yorizori.csv')
    user_info_data = pd.read_csv('./yorizori/yorizori_user.csv').drop(['created_time','updated_time','image_address','nickname','oauth_division'],axis=1)

    target_user_info = user_info_data[user_info_data['user_token_id']==recommend_userid]
    age = target_user_info.iloc[0]["age"]
    gender = target_user_info.iloc[0]["gender"]
    
    
    df_predict = pd.read_csv('./yorizori_predict/yorizori_predict_matrix'+age+'_'+gender+'.csv')
    user_info = pd.read_csv('./yorizori_predict/yorizori_user_values'+age+'_'+gender+'.csv')

    f = open('./yorizori_predict/yorizori_user_index_info'+age+'_'+gender+'.txt','r')
    df_user = f.readlines()
    for i in range(len(df_user)):
        df_user[i]=df_user[i].split("\n")[0]


    mds=md.drop(['created_time','updated_time','authorship','dish_name','recipe_intro','recipe_thumbnail','user_token_id','reference_recipe','level','time'],axis=1)




    #target_user = df_user.index(recommend_userid)
    already_rated, predictions = recommend_recipe(df_predict, recommend_userid, mds, user_info, 50) 
        #예측행렬,예측하는유저id, 레시피테이블 데이터프레임, 유저별 로그 데이터프레임, 몇개추천할지

    
    print(predictions['recipe_id'].to_list())
    predictions

[1002023, 1002341, 1000021, 1000002, 1002476, 1000849, 1001320, 1000890, 1000905, 1000903, 1000902, 1000901, 1000898, 1000896, 1000895, 1000889, 1000908, 1000884, 1001769, 1001775, 1001979, 1001993, 1002007, 1000906, 1000915, 1001246, 1001322, 1001262, 1001277, 1001279, 1001140, 1001233, 1001282, 1001232, 1001335, 1000917, 1001389, 1001033, 1001030, 1001029, 1001394, 1001465, 1000805, 1001467, 1001221, 1000780, 1000071, 1000070, 1001032, 1000886]


In [36]:
predictions

,recipe_id,recipe_view_count,recipe_title,review_count,star_count,version,Predictions
1947,1002023,2351,오징어젓갈,0,0.0,\N,4.354611e-16
2174,1002341,1042,콩국과일펀치,0,0.0,\N,4.354611e-16
18,1000021,5498,카레 재료 카레돼지고기 카레 맛있게 만드는법 카레만들기 단호박카레 방학 점심 메뉴,0,0.0,\N,3.740752e-16
0,1000002,702,고등어초밥,0,0.0,\N,3.493403e-16
2306,1002476,155,치즈떡볶이,0,0.0,\N,3.054245e-16
826,1000849,2997,현미땅콩죽,0,0.0,\N,2.996160e-16
1294,1001320,2732,불고기찹쌀구이,0,0.0,\N,2.996160e-16
866,1000890,1213,모듬초밥,0,0.0,\N,2.683145e-16
881,1000905,2078,유부초밥,0,0.0,\N,2.683145e-16
879,1000903,729,유부쌈밥,0,0.0,\N,2.683145e-16


In [32]:
already_rated

,recipe_id,user_token_id,star_count_x,age,gender,category_value,recipe_view_count_x,values,recipe_view_count_y,recipe_title,review_count,star_count_y,version
0,1000008,test-d80-w20-k10,0,20대 이하,여자,0.409962,0.0,4.099617,2687,초간단 낫또 가지덮밥 만드는 법,0,0.0,\N
1,1000042,test-d80-w20-k10,0,20대 이하,여자,0.409962,0.0,4.099617,1750,대패 간장국수 레시피,0,0.0,\N
2,1000870,test-d80-w20-k10,0,20대 이하,여자,0.409962,0.0,4.099617,957,오므라이스,0,0.0,\N
3,1001248,test-d80-w20-k10,0,20대 이하,여자,0.409962,0.0,4.099617,944,고등어무조림,0,0.0,\N
4,1001295,test-d80-w20-k10,0,20대 이하,여자,0.409962,0.0,4.099617,664,고등어스테이크,0,0.0,\N
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,1002325,test-d80-w20-k10,0,20대 이하,여자,0.409962,0.0,4.099617,1004,장미꽃아이스크림,0,0.0,\N
105,1002328,test-d80-w20-k10,0,20대 이하,여자,0.409962,0.0,4.099617,88,케러멜,0,0.0,\N
106,1002330,test-d80-w20-k10,0,20대 이하,여자,0.409962,0.0,4.099617,1625,트러플쉘,0,0.0,\N
107,1002342,test-d80-w20-k10,0,20대 이하,여자,0.409962,0.0,4.099617,2385,콩국젤리,0,0.0,\N


In [31]:
predictions

,recipe_id,recipe_view_count,recipe_title,review_count,star_count,version,Predictions
0,1000002,702,고등어초밥,0,0.0,\N,0.0
1849,1001925,2726,해초무침,0,0.0,\N,0.0
1824,1001898,1184,시금치샐러드,0,0.0,\N,0.0
1827,1001901,7182,연어샐러드,0,0.0,\N,0.0
1830,1001904,246,젤리샐러드,0,0.0,\N,0.0
1831,1001905,1593,치즈아보카도,0,0.0,\N,0.0
1832,1001906,1235,치즈토마토,0,0.0,\N,0.0
1833,1001907,1386,치킨텐더,0,0.0,\N,0.0
1835,1001909,2932,컵샐러드,0,0.0,\N,0.0
1836,1001910,2019,케이준치킨샐러드,0,0.0,\N,0.0


In [16]:
age=a.iloc[0]["age"]
gender = a.iloc[0]["gender"]
print(age,gender)

20대 이하 여자
